# Scraping for Question 1
Scraping necessary data to look at the impact of the opening kill on the win rate of a round and the different factors of an opening kill.

This data is taken from HLTV.org, which is the premier database for statistics related to the professional Counter-Strike: Global Offensive scene. The data I will be focusing on is taken from all the previous [Majors](https://en.wikipedia.org/wiki/Counter-Strike:_Global_Offensive_Major_Championships). Majors are the tournaments organised by Valve (the developers of CS:GO) that usually occur twice a year, with a prize-pool of $1,000,000 for each event. They are the most prestigious tournaments, similar to how the UEFA Champions League is viewed in European football.

## 1st Scrape: HLTV Scraper
The first step is to import the necessary libraries that will be used in this notebook.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
import time
import re
from loguru import logger
from bs4 import BeautifulSoup

Next, functions are written that will be used later for various purposes.

In [2]:
# Functions

# Scrape Match URLs
def get_urls(soup, urlList):
    # Scrape match URLs from match results page
    for item in soup.find_all(attrs={"class": "date-col"}):
        for link in item.find_all("a"):
            urlList.append("https://www.hltv.org" + link.get("href"))

    # Create URL for next page
    endURL = soup.find(attrs={"class": "pagination-next"}).get("href")
    return urlList, endURL

def get_html_soup(url):
    # Retrieve HTML using headers
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    page = urllib.request.urlopen(req).read()
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(page, "html.parser")
    return soup

def get_full_url(endURL):
    nextURL = "https://www.hltv.org" + endURL
    return nextURL

# Team Functions
def get_event(soup):
    eventBox = soup.find(attrs={"class": "menu-header"})
    event = eventBox.text.strip()
    return event
    
# Match ID
def get_match_ID(soup, URL):
    matchID = URL.split("/")
    return int(matchID[6])

# Date
def get_date(soup):
    dateBox = soup.find(attrs={"class": "small-text"})
    dateTemp = dateBox.text.strip()
    date = dateTemp.split("M")
    return date[0]

# Team Name
def get_team_name_rounds(soup, team):
    if team == 1:
        teamBox = soup.find(attrs={"class": "team-left"})
        teamTemp = teamBox.text.strip()
        team = teamTemp.split("\n")
    elif team == 2:
        teamBox = soup.find(attrs={"class": "team-right"})
        teamTemp = teamBox.text.strip()
        team = teamTemp.split("\n")
        
    else:
        raise ValueError("Team number must be 1 or 2.")
    return team

# Match Winner
def get_match_winner(soup, team1, team2):
    team1Rounds = int(team1[1])
    team2Rounds = int(team2[1])

    if (team1Rounds > team2Rounds):
        matchWinner = team1[0]
    else:
        matchWinner = team2[0]
        
    return matchWinner

# Map
def get_map_pick(soup):
    mapPick = soup.find(attrs={"class":"team-left"}).previous_sibling.strip()
    return mapPick

# Team First Kills, Clutches, and Rating - returns list [int,int,float]
def get_FK_clutches_rating(soup, team):
    teamStatistics = list()
    finalTeamStats = [0,0,0.0]
    for statistic in soup.find_all(attrs={"class":"match-info-row"}):
        teamStatistics.append(statistic.text.strip())
    
    teamFKString = teamStatistics[2]
    teamClutchesString = teamStatistics[3]
    teamRatingString = teamStatistics[1]
    
    teamFKString = teamFKString.split(" ")
    teamClutchesString = teamClutchesString.split(" ")
    teamRatingString = teamRatingString.split(" ")
    
    if team == 1:
        finalTeamStats[0] = int(teamFKString[0])
        finalTeamStats[1] = int(teamClutchesString[0])
        finalTeamStats[2] = float(teamRatingString[0])
    elif team == 2:
        teamFKString = teamFKString[2].split("\\")
        teamClutchesString = teamClutchesString[2].split("\\")
        teamRatingString = teamRatingString[2].split("\\")

        teamFKString = teamFKString[0].split("\n")
        teamClutchesString = teamClutchesString[0].split("\n")
        teamRatingString = teamRatingString[0].split("\n")

        finalTeamStats[0] = int(teamFKString[0])
        finalTeamStats[1] = int(teamClutchesString[0])
        finalTeamStats[2] = float(teamRatingString[0])
        
    else:
        raise ValueError("Team number must be 1 or 2.")
    return finalTeamStats

# Player
# Get player names
def get_player_names(soup):
    playersList = list()
    playerBox = soup.find_all(attrs={"class": "st-player"})

    for x in range(10):
        playersList.append(playerBox[x].text.strip())
        
    return playersList

# Team
def get_player_teams(soup):
    playerTeams = list()
    playerTeamsBox = soup.find_all(attrs={"class": "st-teamname text-ellipsis"})
    playerTeams.append(playerTeamsBox[0].text.strip())
    playerTeams.append(playerTeamsBox[1].text.strip())
    
    return playerTeams

# Get player kills
def get_player_kills(soup):
    killList = list()
    killBox = soup.find_all(attrs={"class": "st-kills"})
    # Team 1
    for x in range(1, 6):
        killStrip = killBox[x].text.strip()
        killStrip = killStrip.split(" ")
        killList.append(int(killStrip[0]))
    # Team 2
    for x in range(7, 12):
        killStrip = killBox[x].text.strip()
        killStrip = killStrip.split(" ")
        killList.append(int(killStrip[0]))
        
    return killList

# Get player deaths
def get_player_deaths(soup):
    deathList = list()
    deathBox = soup.find_all(attrs={"class": "st-deaths"})
    # Team 1
    for x in range(1, 6):
        deaths = deathBox[x].text.strip()
        deathList.append(int(deaths))
    # Team 2
    for x in range(7, 12):
        deaths = deathBox[x].text.strip()
        deathList.append(int(deaths))
        
    return deathList

# Get player first kills
def get_player_FK(soup):
    firstKillList = list()
    firstDeathList = list()
    firstKDBox = soup.find_all(attrs={"class": "st-fkdiff"})

    # Team 1
    for x in range(1, 6):
        firstKDString = str(firstKDBox[x])
        firstKDString = firstKDString.split("=\"")
        kdDifference = firstKDString[4]
        kdDifference = kdDifference.split("\"")
        kdSplit = kdDifference[0].split(" ")
        firstKillList.append(int(kdSplit[0]))
    # Team 2
    for x in range(7, 12):
        firstKDString = str(firstKDBox[x])
        firstKDString = firstKDString.split("=\"")
        kdDifference = firstKDString[4]
        kdDifference = kdDifference.split("\"")
        kdSplit = kdDifference[0].split(" ")
        firstKillList.append(int(kdSplit[0]))
        
    return firstKillList

# Get player first deaths
def get_player_FD(soup):
    firstKillList = list()
    firstDeathList = list()
    firstKDBox = soup.find_all(attrs={"class": "st-fkdiff"})

    # Team 1
    for x in range(1, 6):
        firstKDString = str(firstKDBox[x])
        firstKDString = firstKDString.split("=\"")
        kdDifference = firstKDString[4]
        kdDifference = kdDifference.split("\"")
        kdSplit = kdDifference[0].split(" ")
        firstDeathList.append(int(kdSplit[3]))
    # Team 2
    for x in range(7, 12):
        firstKDString = str(firstKDBox[x])
        firstKDString = firstKDString.split("=\"")
        kdDifference = firstKDString[4]
        kdDifference = kdDifference.split("\"")
        kdSplit = kdDifference[0].split(" ")
        firstDeathList.append(int(kdSplit[3]))
        
    return firstDeathList

# Rating
def get_player_ratings(soup):
    ratingList = list()
    ratingBox = soup.find_all(attrs={"class": "st-rating"})
    # Team 1
    for x in range(1, 6):
        ratings = ratingBox[x].text.strip()
        ratingList.append(float(ratings))
    # Team 2
    for x in range(7, 12):
        ratings = ratingBox[x].text.strip()
        ratingList.append(float(ratings))

    return ratingList

To scrape the data from the specified matches, a list of URLs is needed. These URLs are located in the Major section of the [HLTV results page](https://www.hltv.org/stats/matches?matchType=Majors). The build a list of these URLs, the results page will be scraped. An example of this page is downloaded below. As HLTV is CloudFlare protected, a header parameter is added to the urllib.request to bypass the 403 error returned. A sleep is also used, as too many requests too quickly can result in an IP ban.

In [3]:
# Download example HTML to understand format
# Build opener to bypass 403 error
opener = urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3')]
urllib.request.install_opener(opener)

url = "https://www.hltv.org/stats/matches?matchType=Majors"
local = "EXAMPLE_HTML_1.html"

# Download example HTML file, save as example.html
urllib.request.urlretrieve(url,local)

# Sleep for 3 seconds
time.sleep(2)

After reviewing the example HTML, I was able to determine where the match URLs were stored, but also the link to the next page of Major matches. There are a total of 622 Major matches, with only 50 being shown at a time per page. On the final page of match results, the next page button returns a None value. This allowed me to run a while loop which would continue until the last page of the results.

Using BeatifulSoup to parse HTML, a results page is scraped for its match links, which are in the format "stats/matches/mapstatsid/MATCHID/TEAM1-vs-TEAM2?matchType=Majors". When combined with the base URL "https://www.hltv.org", a full URL is created, which is appended to a DataFrame and used for scraping later. This method is used again for the next page URL. The header and sleep technique used previously are again used here.

Intially, the example HTML is scraped, collecting the first 50 match URLs and the next page URL. In the next cell, the rest of the URLs are collected in the while loop.

In [4]:
# List for match URLS
majorURLList = list()

# Parse HTML of example HTML downloaded
soup = BeautifulSoup(open("EXAMPLE_HTML_1.html", encoding="utf8"), "html.parser")

# Scrape match URLs from match results page
majorURLList, endURL = get_urls(soup, majorURLList)

# Form URL
nextURL = get_full_url(endURL)

Using the first next page URL scraped, iterate until the next page value is None. The list of URLs is printed after completion.

In [5]:
# While the next page returns a value, scrape
while(endURL != None):
    # Logger message
    logger.info('Scraping new event page: %s' % nextURL)
    
    # Using urllib and BeautifulSoup, parse HTML page
    soup = get_html_soup(nextURL)
    
    # Scrape match URLs from match results page
    majorURLList, endURL = get_urls(soup, majorURLList)
    
    # Check to see if value returned is None, if not form next URL
    if endURL != None:
        nextURL = nextURL = get_full_url(endURL)

# Return list
majorURLList

2019-05-10 19:19:01.734 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=50


2019-05-10 19:19:04.289 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=100


2019-05-10 19:19:06.839 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=150


2019-05-10 19:19:09.449 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=200


2019-05-10 19:19:12.015 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=250


...

2019-05-10 19:19:22.424 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=450


2019-05-10 19:19:25.037 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=500


2019-05-10 19:19:27.599 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=550


2019-05-10 19:19:30.152 | INFO     | __main__:<module>:4 - Scraping new event page: https://www.hltv.org/stats/matches?matchType=Majors&offset=600


['https://www.hltv.org/stats/matches/mapstatsid/82145/ence-vs-astralis?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82143/astralis-vs-ence?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82123/mibr-vs-astralis?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82121/astralis-vs-mibr?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82120/natus-vincere-vs-ence?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82119/ence-vs-natus-vincere?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82117/natus-vincere-vs-ence?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82095/nip-vs-astralis?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82091/astralis-vs-nip?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82087/renegades-vs-mibr?matchType=Majors',
 'https://www.hltv.org/stats/matches/mapstatsid/82082/mibr-vs-renegades?matchType=Majo

After successfully scraping the URLs, the necessary statistics are now scraped from the individual matches. Two DataFrames are created, one for the overall match statistics and one for individual player statistics for each match. In these DataFrames first kills are abbrieviated to FK, first deaths to FD, percentage of kills that were first kills to FK%K, and percentage of deaths that were first deaths to FK%D.

In [6]:
# Match statistics
dfMatchStats = pd.DataFrame(columns=["Event", "Date", "Match ID", "Team 1", "Team 2", "Map", "Team 1 Rounds", 
                                     "Team 2 Rounds", "Total Rounds", "Winner", "Team 1 FK", "Team 2 FK", 
                                     "Team 1 Clutches", "Team 2 Clutches", "Team 1 Rating", "Team 2 Rating"])

# Player statistics
dfPlayerStats = pd.DataFrame(columns=["Event", "Date", "Match ID", "Player", "Team", "Map", "Rounds", "Kills", 
                                      "Deaths", "FK", "FD", "FK%K", "FD%D", "Rating"])

For each URL, the same method of scraping is used that was used previously. Using the functions declared earlier, each HTML is scraped for the statistics needed and the data is appeneded to the corresponding DataFrame.

In [7]:
# For loop to scrape the data of each major match
for URL in majorURLList:
    # Logger message
    logger.info('Scraping new match: %s' % URL)
    
    # Retrieve HTML and parse with BeautifulSoup
    soup = get_html_soup(URL)
    
    # Get info
    # Team
    event = get_event(soup)
    matchID = get_match_ID(soup, URL)
    date = get_date(soup)
    team1 = get_team_name_rounds(soup, 1)
    team2 = get_team_name_rounds(soup, 2)
    matchWinner = get_match_winner(soup, team1, team2)
    mapPick = get_map_pick(soup)
    team1FK = get_FK_clutches_rating(soup, 1)
    team2FK = get_FK_clutches_rating(soup, 2)

    # Player
    playerNames = get_player_names(soup)
    playerTeams = get_player_teams(soup)
    playerKills = get_player_kills(soup)
    playerDeaths = get_player_deaths(soup)
    playerFK = get_player_FK(soup)
    playerFD = get_player_FD(soup)
    playerRatings = get_player_ratings(soup)

    # Append Match Stats
    dfMatchStats = dfMatchStats.append({"Event": event, "Date": date, "Match ID": matchID, "Team 1": team1[0], 
                                        "Team 1 Rounds": int(team1[1]), "Team 2": team2[0], 
                                        "Team 2 Rounds": int(team2[1]), "Total Rounds": int(team1[1]) + int(team2[1]), 
                                        "Winner": matchWinner, "Map": mapPick, "Team 1 FK": team1FK[0], 
                                        "Team 1 Clutches": team1FK[1],  "Team 1 Rating": team1FK[2], 
                                        "Team 2 FK": team2FK[0],  "Team 2 Clutches": team2FK[1], 
                                        "Team 2 Rating": team2FK[2]}, ignore_index=True)


    #Append Team 1
    for x in range(5):
        dfPlayerStats = dfPlayerStats.append({"Event": event, "Date": date, "Match ID": matchID, "Team": playerTeams[0], 
                                              "Map": mapPick, "Rounds": int(team1[1]) + int(team2[1]), 
                                              "Player": playerNames[x], "Kills": playerKills[x], 
                                              "Deaths": playerDeaths[x], "FK": playerFK[x], "FD": playerFD[x], 
                                              "FK%K": (playerFK[x]/playerKills[x])*100, 
                                              "FD%D": (playerFD[x]/playerDeaths[x])*100, 
                                              "Rating": playerRatings[x]}, ignore_index=True)

    # Append Team 2
    for x in range(5, 10):
        dfPlayerStats = dfPlayerStats.append({"Event": event, "Date": date, "Match ID": matchID, "Team": playerTeams[1], 
                                              "Map": mapPick, "Rounds": int(team1[1]) + int(team2[1]), 
                                              "Player": playerNames[x], "Kills": playerKills[x], 
                                              "Deaths": playerDeaths[x], "FK": playerFK[x], "FD": playerFD[x], 
                                              "FK%K": (playerFK[x]/playerKills[x])*100, 
                                              "FD%D": (playerFD[x]/playerDeaths[x])*100, 
                                              "Rating": playerRatings[x]}, ignore_index=True)


2019-05-10 19:19:32.931 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/82145/ence-vs-astralis?matchType=Majors


2019-05-10 19:19:35.392 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/82143/astralis-vs-ence?matchType=Majors


2019-05-10 19:19:38.031 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/82123/mibr-vs-astralis?matchType=Majors


2019-05-10 19:19:40.617 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/82121/astralis-vs-mibr?matchType=Majors


2019-05-10 19:19:43.177 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/82120/natus-vincere-vs-ence?matchType=Majors


...

2019-05-10 19:46:07.309 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/15819/lgb-vs-clan-mystik?matchType=Majors


2019-05-10 19:46:09.829 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/15818/clan-mystik-vs-fnatic?matchType=Majors


2019-05-10 19:46:12.359 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/15817/lgb-vs-natus-vincere?matchType=Majors


2019-05-10 19:46:14.899 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/15816/natus-vincere-vs-fnatic?matchType=Majors


2019-05-10 19:46:17.404 | INFO     | __main__:<module>:4 - Scraping new match: https://www.hltv.org/stats/matches/mapstatsid/15815/clan-mystik-vs-lgb?matchType=Majors


Some simple cleaning is undertaken on the DataFrames. The Date column of each DataFrame is converted to DateTime format and the Match ID column is set as the index.

In [8]:
# Convert Date to DateTime data type
dfMatchStats["Date"] =  pd.to_datetime(dfMatchStats["Date"], format="%Y-%m-%d %H:%M")

# Set Match ID as index
dfMatchStats.set_index("Match ID", inplace=True)
        
# Convert Date to DateTime data type
dfPlayerStats["Date"] =  pd.to_datetime(dfPlayerStats["Date"], format="%Y-%m-%d %H:%M")

# Set Match ID as index
dfPlayerStats.set_index("Match ID", inplace=True)

The DataFrames are then checked to see if there are any null values.

In [9]:
dfMatchStats.isnull().sum()

Event              0
Date               0
Team 1             0
Team 2             0
Map                0
Team 1 Rounds      0
Team 2 Rounds      0
Total Rounds       0
Winner             0
Team 1 FK          0
Team 2 FK          0
Team 1 Clutches    0
Team 2 Clutches    0
Team 1 Rating      0
Team 2 Rating      0
dtype: int64

In [10]:
dfPlayerStats.isnull().sum()

Event     0
Date      0
Player    0
Team      0
Map       0
Rounds    0
Kills     0
Deaths    0
FK        0
FD        0
FK%K      0
FD%D      0
Rating    0
dtype: int64

A sample of each DataFrame is printed to give an example of the format.

In [11]:
dfMatchStats.sample(5)

,Event,Date,Team 1,Team 2,Map,Team 1 Rounds,Team 2 Rounds,Total Rounds,Winner,Team 1 FK,Team 2 FK,Team 1 Clutches,Team 2 Clutches,Team 1 Rating,Team 2 Rating
Match ID,,,,,,,,,,,,,,,
73726,FACEIT Major 2018,2018-09-13 17:50:00,Natus Vincere,FaZe,Overpass,16,12,28,Natus Vincere,15,13,2,1,1.12,1.00
59932,ELEAGUE Major 2018,2018-01-19 20:10:00,Astralis,mousesports,Cache,2,16,18,mousesports,7,11,0,2,0.58,1.60
18729,DreamHack Winter 2014,2014-11-29 21:00:00,NiP,LDLC,Overpass,16,19,35,LDLC,15,20,2,3,0.99,1.02
17553,ESL One Cologne 2014,2014-08-16 15:45:00,LDLC,Virtus.pro,Mirage,16,12,28,LDLC,16,12,2,1,1.12,0.84
82095,IEM Katowice 2019,2019-03-01 18:35:00,Astralis,NiP,Dust2,16,14,30,Astralis,19,11,1,0,1.07,0.97


In [12]:
dfPlayerStats.sample(5)

,Event,Date,Player,Team,Map,Rounds,Kills,Deaths,FK,FD,FK%K,FD%D,Rating
Match ID,,,,,,,,,,,,,
60219,ELEAGUE Major 2018,2018-01-28 19:15:00,rain,FaZe,Inferno,41,17,30,5,7,29.411765,23.333333,0.74
49710,PGL Major Krakow 2017,2017-07-17 16:10:00,tabseN,BIG,Inferno,27,28,15,4,2,14.285714,13.333333,1.65
15827,DreamHack Winter 2013,2013-11-28 22:30:00,kennyS,Recursive,Train,26,12,23,1,2,8.333333,8.695652,0.50
60196,ELEAGUE Major 2018,2018-01-27 23:25:00,tarik,Cloud9,Mirage,19,13,12,2,3,15.384615,25.000000,1.33
16497,EMS One Katowice 2014,2014-03-14 00:40:00,JW,fnatic,Inferno,22,16,13,4,2,25.000000,15.384615,1.09


The list of Major match URLs is converted to a DataFrame and the three DataFrames are converted to CSV files and saved locally.

In [13]:
dfHLTVURLs = pd.DataFrame(majorURLList)

In [14]:
dfMatchStats.to_csv("MAJOR_MATCH_STATS.csv", header=True, index=True)
dfPlayerStats.to_csv("MAJOR_PLAYER_STATS.csv", header=True, index=True)
dfHLTVURLs.to_csv("HLTV_URLS.csv", header=False, index=True)